In [ ]:
import warnings
warnings.filterwarnings('always')
from keras_tensorboard import plot_all_plots

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, auc, roc_curve
import keras
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Input, TimeDistributed, Masking, Dropout
from keras.layers import Dense, Flatten, MaxPooling2D, Convolution2D
from keras.optimizers import Adam
from keras.regularizers import l1, l2
from keras.callbacks import Callback, LambdaCallback
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.utils import to_categorical

In [ ]:
seed = 42
np.random.seed(seed)
np.set_printoptions(precision=2)

In [ ]:
# Placeholder for data
class DataPlaceholder():
    def __init__(self):
        class DataItem():
            def __init__(self):
                self.x = None
                self.y = None
        self.train = DataItem()
        self.val = DataItem()
        self.test = DataItem()
    def print_shapes(self):
        print('Train shapes: x={} , y={}'.format(data.train.x.shape,data.train.y.shape))
        print('Val shapes  : x={} , y={}'.format(data.val.x.shape,data.val.y.shape))
        print('Test shapes : x={} , y={}'.format(data.test.x.shape,data.test.y.shape))

# Data prediction Callback

In [ ]:
class PredictData(Callback):
    def __init__(self,model,x,y,log_word):
        self.x = x; self.y = y; 
        self.model = model
        self.log_word = log_word

    def on_epoch_end(self,epoch,logs={}):
        logs[self.log_word+'predictions'] = self.model.predict(self.x)
        logs[self.log_word+'labels'] = self.y

# Scalars Dataset

In [ ]:
from sklearn.datasets import make_regression
data = DataPlaceholder()
X, y = make_regression(n_samples=100, n_features=5, noise=0.1)
y = (y - y.mean())/y.std()
X_left, data.test.x, y_left, data.test.y = train_test_split(X,y,test_size=0.2)
data.train.x, data.val.x, data.train.y, data.val.y = train_test_split(X_left,y_left,test_size=0.3)
data.print_shapes()

## Metrics

In [ ]:
def mse(y_true,y_pred):
    return np.mean( (y_pred-y_true)**2, axis=None )

def std(y_true,y_pred):
    return np.std( (y_pred-y_true), axis=None )

metrics_functions = {
    'mse': mse,
    'error_std': std
}

## Simple Scalar

In [ ]:
# model 1
model = Sequential()
model.add(Dense(2, activation='relu', input_dim=5))
model.add(Dense(1))
model.compile(loss='mse',optimizer='adam')

callbacks = [
    PredictData(model, x=data.train.x, y=data.train.y, log_word=''),
    PredictData(model, x=data.val.x  , y=data.val.y  , log_word='val_'),
    PredictData(model, x=data.test.x , y=data.test.y , log_word='test_'),
]
h = model.fit( data.train.x, data.train.y, epochs=30, batch_size=4, verbose=0,
                validation_data=(data.val.x,data.val.y), callbacks=callbacks)

In [ ]:
# model 2
model = Sequential()
model.add(Dense(2, activation='relu', input_dim=5))
model.add(Dense(1))
model.compile(loss='mse',optimizer='sgd')

callbacks = [
    PredictData(model, x=data.train.x, y=data.train.y, log_word=''),
    PredictData(model, x=data.val.x  , y=data.val.y  , log_word='val_'),
    PredictData(model, x=data.test.x , y=data.test.y , log_word='test_'),
]
h2 = model.fit( data.train.x, data.train.y, epochs=30, batch_size=4, verbose=0,
                validation_data=(data.val.x,data.val.y), callbacks=callbacks)

In [ ]:
models = {
    'model1':h.history,
    'model2':h2.history
}
plot_all_plots('/tmp/example',models,metrics_functions)

## K-Folded Scalar

In [ ]:
# model 1
model = Sequential()
model.add(Dense(2, activation='relu', input_dim=5))
model.add(Dense(1))
model.compile(loss='mse',optimizer='adam')

ids = np.arange( len(data.train.x) + len(data.val.x) )
histories = []
for idx_train, idx_valid in KFold(n_splits=5,shuffle=True).split(ids):
    data.train.x, data.val.x = X[ idx_train ], X[ idx_valid ]
    data.train.y, data.val.y = y[ idx_train ], y[ idx_valid ]
    callbacks = [
        PredictData(model, x=data.train.x, y=data.train.y, log_word=''),
        PredictData(model, x=data.val.x  , y=data.val.y  , log_word='val_'),
        PredictData(model, x=data.test.x , y=data.test.y , log_word='test_'),
    ]
    h = model.fit( data.train.x, data.train.y, epochs=30, batch_size=4, verbose=0,
                   validation_data=(data.val.x,data.val.y), callbacks=callbacks)
    histories.append(h.history)

In [ ]:
# model 2
model = Sequential()
model.add(Dense(2, activation='relu', input_dim=5))
model.add(Dense(1))
model.compile(loss='mse',optimizer='sgd')

ids = np.arange( len(data.train.x) + len(data.val.x) )
histories2 = []
for idx_train, idx_valid in KFold(n_splits=5,shuffle=True).split(ids):
    data.train.x, data.val.x = X[ idx_train ], X[ idx_valid ]
    data.train.y, data.val.y = y[ idx_train ], y[ idx_valid ]
    callbacks = [
        PredictData(model, x=data.train.x, y=data.train.y, log_word=''),
        PredictData(model, x=data.val.x  , y=data.val.y  , log_word='val_'),
        PredictData(model, x=data.test.x , y=data.test.y , log_word='test_'),
    ]
    h = model.fit( data.train.x, data.train.y, epochs=30, batch_size=4, verbose=0,
                   validation_data=(data.val.x,data.val.y), callbacks=callbacks)
    histories2.append(h.history)

In [ ]:
models = {
    'model1':histories,
    'model2':histories2
}
plot_all_plots('/tmp/example',models,metrics_functions,kfolded=True)

# Multi-Class Dataset

In [ ]:
from sklearn.datasets import make_blobs
data = DataPlaceholder()
X, y = make_blobs(n_samples=100, centers=3, n_features=5)
y = to_categorical(y)
X_left, data.test.x, y_left, data.test.y = train_test_split(X,y,test_size=0.2)
data.train.x, data.val.x, data.train.y, data.val.y = train_test_split(X_left,y_left,test_size=0.3)
data.print_shapes()

In [ ]:
index2class_name = {
    0:'car',
    1:'bike',
    2:'plain'
}

## Metrics

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, \
                            f1_score, recall_score, precision_score

def accuracy(y_true,y_pred):
    num_classes = y_true.shape[-1]
    y_argmax = np.argmax(y_pred,axis=1)
    y_pred = to_categorical(y_argmax,num_classes=num_classes)
    accs = np.zeros(num_classes)
    print(y_true[:10])
    print(y_pred[:10])
    print(y_pred.shape,y_true.shape)
    for i in range(num_classes):
        accs[i] = accuracy_score(y_true[:,i],y_pred[:,i])
    return accs

def weighted_accuracy(weights):
    weights = np.array(weights)
    def acc(y_true,y_pred):
        W = weights[ np.argmax(y_true,axis=1) ]
        return np.mean( W * (np.argmax(y_true,axis=1) == np.argmax(y_pred,axis=1)) )
    return acc

def recall(y_true,y_pred):
    num_classes = y_true.shape[-1]
    y_argmax = np.argmax(y_pred,axis=1)
    y_pred = to_categorical(y_argmax,num_classes=num_classes)
    return recall_score(y_true,y_pred,average=None)

def precision(y_true,y_pred):
    num_classes = y_true.shape[-1]
    y_argmax = np.argmax(y_pred,axis=1)
    y_pred = to_categorical(y_argmax,num_classes=num_classes)
    return precision_score(y_true,y_pred,average=None)

metrics_functions = {
    'accuracy': accuracy,
    'weighted_acc': weighted_accuracy([3,0.1,21]),
    'recall':recall,
    'precision':precision
}

## Simple Multi-Class

In [ ]:
# model 1
model = Sequential()
model.add(Dense(2, activation='relu', input_dim=5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='mse',optimizer='adam')

callbacks = [
    PredictData(model, x=data.train.x, y=data.train.y, log_word=''),
    PredictData(model, x=data.val.x  , y=data.val.y  , log_word='val_'),
    PredictData(model, x=data.test.x , y=data.test.y , log_word='test_'),
]
h = model.fit( data.train.x, data.train.y, epochs=30, batch_size=4, verbose=0,
               validation_data=(data.val.x,data.val.y), callbacks=callbacks)

In [ ]:
# model 2
model = Sequential()
model.add(Dense(2, activation='relu', input_dim=5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='mse',optimizer='sgd')

callbacks = [
    PredictData(model, x=data.train.x, y=data.train.y, log_word=''),
    PredictData(model, x=data.val.x  , y=data.val.y  , log_word='val_'),
    PredictData(model, x=data.test.x , y=data.test.y , log_word='test_'),
]
h2 = model.fit( data.train.x, data.train.y, epochs=30, batch_size=4, verbose=0,
               validation_data=(data.val.x,data.val.y), callbacks=callbacks)

In [ ]:
models = {
    'model1':h.history,
    'model2':h2.history
}
plot_all_plots('/tmp/example',models,metrics_functions,kfolded=False,pr_curve=True)

## K-Folded Multi-Class

In [ ]:
# model 1
model = Sequential()
model.add(Dense(2, activation='relu', input_dim=5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='mse',optimizer='adam')

ids = np.arange( len(data.train.x) + len(data.val.x) )
histories = []
for idx_train, idx_valid in KFold(n_splits=5,shuffle=True).split(ids):
    data.train.x, data.val.x = X[ idx_train ], X[ idx_valid ]
    data.train.y, data.val.y = y[ idx_train ], y[ idx_valid ]
    callbacks = [
        PredictData(model, x=data.train.x, y=data.train.y, log_word=''),
        PredictData(model, x=data.val.x  , y=data.val.y  , log_word='val_'),
        PredictData(model, x=data.test.x , y=data.test.y , log_word='test_'),
    ]
    h = model.fit( data.train.x, data.train.y, epochs=30, batch_size=4, verbose=0,
                   validation_data=(data.val.x,data.val.y), callbacks=callbacks)
    histories.append(h.history)

In [ ]:
# model 2
model = Sequential()
model.add(Dense(2, activation='relu', input_dim=5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='mse',optimizer='sgd')

ids = np.arange( len(data.train.x) + len(data.val.x) )
histories2 = []
for idx_train, idx_valid in KFold(n_splits=5,shuffle=True).split(ids):
    data.train.x, data.val.x = X[ idx_train ], X[ idx_valid ]
    data.train.y, data.val.y = y[ idx_train ], y[ idx_valid ]
    callbacks = [
        PredictData(model, x=data.train.x, y=data.train.y, log_word=''),
        PredictData(model, x=data.val.x  , y=data.val.y  , log_word='val_'),
        PredictData(model, x=data.test.x , y=data.test.y , log_word='test_'),
    ]
    h = model.fit( data.train.x, data.train.y, epochs=30, batch_size=4, verbose=0,
                   validation_data=(data.val.x,data.val.y), callbacks=callbacks)
    histories2.append(h.history)

In [ ]:
models = {
    'model1':histories,
    'model2':histories
}
plot_all_plots('/tmp/multi_kfold_example',models,metrics_functions,index2class_name,kfolded=True,pr_curve=True)